<h2>What is Chat-GPT?</h2>

With the recent release of Chat GPT-4, many people are enthralled by the programs capabilities, and many are also expressing concerns regarding the implications that come with releasing a powerful AI open to use to anyone and everyone who desire it. What exactly is Chat-GPT? Chat GPT is an Artificial Intelligence chat bot developed and released by OpenAI, a research team of scientists and engineers working to further develop the capabilities of artificial intelligence. According to OpenAI Chat-GPT is powered by a deep learning model, which is a type of neural network designed to process and generate language. GPT learns by being fed large amounts of data and text which it processes and ‘learns’ to discuss and generate answers about that data in the same context in which it took in that data. 

<h2>Why is it important to discuss?</h2>

The advancement of powerful natural language processing technology has come rapidly, opening up possibilities for its users and empowering people to do more with their computers. Chat-GPT 4 has been used to aid education, programming work, day-to-day inquiries, and can even be used for data analysis. Recently, the AI powering Chat GPT-4 has been used to code entire programs, and even create falsified videos and audio files. While the capabilities of the new AI are certainly incredible, its release to the public in the form of a chat-bot has also caused concern amongst many. Some concerns include its unlimited access to information that can out match a human counterpart easily, and scifi-like concerns about its ability to become more powerful than humans and overpower us. While AI sounds like something that is bound to a computer screen, many governments, such as Germany, have started drafting legislation in response to the development of AI’s such as Chat-GPT 4. Powerful AIs, such as Chat-GPT are here and how it is perceived and used everyday will dictate how we move forward with the new technology. 


<h2>Purpose</h2>

The Purpose of this tutorial is to analyze the sentiments surrounding recent advancements in AI technology and determine the general attitude towards AI technology, such as Chat-GPT. Understanding this sentiment is crucial in guiding our future course of action for the development and release of subsequent AI tools, as well as the formulations of relevant policies that aim to mitigate potential risks associated with AI


<h2>Data Collection</h2>

With Chat-GPT arguably being one of the most popular and most discussed releases of AI software in recent history we decided to center our data collection around tweets from the popular social media platform Twitter. Specifically, we focused on tweets containing the hashtag '#ChatGPT' starting from the initial launch of the OpenAI chatbot in November 2022 until April 7th, 2023. Our analysis revolves around the tweet's contents, the number of likes it received, and the number of retweets it accumulated. By assessing the overall sentiment of each tweet as either positive or negative, we aim to determine the general perception of Twitter users regarding Chat-GPT.

<h2>Imports</h2>

In [18]:
#Add imports
import pandas as pd
import numpy as np
from textblob import TextBlob as tb

In [11]:
#read tweets into dataframe
df = pd.read_csv("archive/ChatGPT.csv",low_memory=False)

#remove unneccessary columns (subject to change)
df.drop(columns=['Unnamed: 0', 'Location', 'Description'], axis=1, inplace=True)

In [21]:
#remove time information from date column
df['Date'] = df['Date'].astype(str)
df['Date'] = df['Date'].str.extract(r'(\d{4}-\d{2}-\d{2})')

df.head()

,Date,Tweet,Url,User,UserCreated,UserVerified,UserFollowers,UserFriends,Retweets,Likes
0,2023-04-08,#ChatGPT If only there were a movie or somethi...,https://twitter.com/kirbydjohnson/status/16445...,kirbydjohnson,2012-07-29 22:57:46+00:00,False,50.0,76.0,0,0
1,2023-04-08,"Workshop Description: In this workshop, you wi...",https://twitter.com/UniversityWeb/status/16445...,UniversityWeb,2013-01-13 20:18:16+00:00,False,9056.0,387.0,0,0
2,2023-04-08,New Post at AiNewsDrop!\n\nChatGPT is a doctor...,https://twitter.com/AiNewsDrop/status/16445436...,AiNewsDrop,2022-01-19 06:25:45+00:00,False,276.0,313.0,0,0
3,2023-04-08,OpenAI’s GPT-4 Just Got Supercharged! #ai #Cha...,https://twitter.com/tubeblogger/status/1644543...,tubeblogger,2013-06-25 01:42:59+00:00,False,312.0,293.0,0,0
4,2023-04-08,"""Classical art"" is struggling - not changed th...",https://twitter.com/majorradic/status/16445432...,majorradic,2011-12-12 01:29:15+00:00,False,522.0,684.0,0,2


In [25]:
#create fucntion using textblob to gauge attitudes
def get_attitudes(tweet):
    result = tb(tweet)
    attitude = result.sentiment.polarity
    return attitude

In [27]:
#get the attitude score for each tweet
df['Tweet'] = df['Tweet'].astype(str)
df['Attitudes'] = df['Tweet'].apply(get_attitudes)
df.head()

,Date,Tweet,Url,User,UserCreated,UserVerified,UserFollowers,UserFriends,Retweets,Likes,Attitudes
0,2023-04-08,#ChatGPT If only there were a movie or somethi...,https://twitter.com/kirbydjohnson/status/16445...,kirbydjohnson,2012-07-29 22:57:46+00:00,False,50.0,76.0,0,0,0.000000
1,2023-04-08,"Workshop Description: In this workshop, you wi...",https://twitter.com/UniversityWeb/status/16445...,UniversityWeb,2013-01-13 20:18:16+00:00,False,9056.0,387.0,0,0,0.466667
2,2023-04-08,New Post at AiNewsDrop!\n\nChatGPT is a doctor...,https://twitter.com/AiNewsDrop/status/16445436...,AiNewsDrop,2022-01-19 06:25:45+00:00,False,276.0,313.0,0,0,0.170455
3,2023-04-08,OpenAI’s GPT-4 Just Got Supercharged! #ai #Cha...,https://twitter.com/tubeblogger/status/1644543...,tubeblogger,2013-06-25 01:42:59+00:00,False,312.0,293.0,0,0,0.000000
4,2023-04-08,"""Classical art"" is struggling - not changed th...",https://twitter.com/majorradic/status/16445432...,majorradic,2011-12-12 01:29:15+00:00,False,522.0,684.0,0,2,0.200000


In [ ]:
# Define weights to likes and retweets
like_weight = 0.5
rt_weight = 0.2

#calculated weighted scores
popularity = pd.DataFrame()
popularity['weighted_likes'] = df['Likes'] * like_weight
popularity['weighted_rt'] = df['rt'] * rt_weight

#get overall popularity score
df['Popularity_Score'] = popularity['weighted_likes'] + popularity['weighted_rt']

# Calculate the weighted score by multiplying the attitude score with the popularity score
df['weighted_score'] = df['Attitudes'] * df['Popularity_Score']
